In [1]:
import requests
import json

import pandas as pd
import numpy as np

# Data Frame

In [2]:
HTTPS = 'https://'
DOMAIN = 'api.laligafantasymarca.com'

# V1
V1 = '/stats/v1'
HTTPS_DOMAIN_V1 = f'{HTTPS}{DOMAIN}{V1}'
ROUTE_WEEK = lambda week: f'/stats/week/{week}'

# V3
V3 = '/api/v3'
HTTPS_DOMAIN_V3 = f'{HTTPS}{DOMAIN}{V3}'
ROUTE_PLAYERS = '/players'
ROUTE_MARKET_VALUE = lambda id: f'/player/{id}/market-value'
ROUTE_PLAYER_STATS = lambda id: f'/player/{id}'


In [3]:
# get list of players ids
response = requests.get(f'{HTTPS_DOMAIN_V3}{ROUTE_PLAYERS}')

players = response.json()
players_ids = [p['id'] for p in players]
players_path = './data_json/players.json'


with open(players_path, 'w') as convert_file:
     convert_file.write(json.dumps(players))



In [7]:
{p['nickname']: p for p in players[1:10]}.keys()

dict_keys(['Unai Simón', 'Ter Stegen', 'Iván Villar', 'David Soria', 'Aitor Fdez.', 'Courtois', 'Remiro', 'Jaume Domenech', 'Asenjo'])

## Player Stats

In [209]:
player_stats_path = './data_json/player_stats.json'

### Get data from local JSON

In [176]:
# with open(player_stats_path) as f:
#     players_stats = json.load(f)

### Get data from API

In [210]:
# get player stats
print('Getting player stats...')

# get players stats info for each player
players_stats = []
for i, p_id in enumerate(players_ids):
    response = requests.get(f'{HTTPS_DOMAIN_V3}{ROUTE_PLAYER_STATS(p_id)}')
    p_stats = response.json()

    if p_stats['position'] != 'Entrenador':
        players_stats.append(p_stats)
    
    print(i+1, '- Retrieved player stats of id:', p_id)
    
# create file
with open(player_stats_path, 'w') as convert_file:
     convert_file.write(json.dumps(players_stats))

len(players_stats)

Getting player stats...
1 - Retrieved player stats of id: 53
2 - Retrieved player stats of id: 68
3 - Retrieved player stats of id: 93
4 - Retrieved player stats of id: 140
5 - Retrieved player stats of id: 184
6 - Retrieved player stats of id: 204
7 - Retrieved player stats of id: 230
8 - Retrieved player stats of id: 274
9 - Retrieved player stats of id: 298
10 - Retrieved player stats of id: 339
11 - Retrieved player stats of id: 362
12 - Retrieved player stats of id: 644
13 - Retrieved player stats of id: 677
14 - Retrieved player stats of id: 734
15 - Retrieved player stats of id: 804
16 - Retrieved player stats of id: 844
17 - Retrieved player stats of id: 905
18 - Retrieved player stats of id: 908
19 - Retrieved player stats of id: 947
20 - Retrieved player stats of id: 993
21 - Retrieved player stats of id: 1046
22 - Retrieved player stats of id: 1066
23 - Retrieved player stats of id: 1100
24 - Retrieved player stats of id: 1132
25 - Retrieved player stats of id: 1138
26 - Ret

509

### Create data frame

In [178]:
# # normalize player stats and create data frame
# df_players_stats = pd.json_normalize(players_stats, 
#                                         record_path=['playerStats'],
#                                         meta=['team', 'id', 'name', 'position'],
#                                         errors=None)

df_players_stats = pd.json_normalize(players_stats, 
                                        record_path=['playerStats'],
                                        meta=['team', 'id', 'name', 'position'],
                                        errors='raise')

In [179]:
df_players_stats.shape

(8751, 27)

### Create column for each of the team information (teamName, teamId)

In [180]:
# function to get team data
def flatten_team(d):
    return {'team_id': d['id'], 'team_shortName': d['shortName']}

# apply function to extract info
df_players_stats[['team_id', 'team_shortName']] = df_players_stats['team'].apply(lambda d: pd.Series(flatten_team(d)))

# drop team column
df_players_stats = df_players_stats.drop(columns='team')

In [181]:
df_players_stats.head()

,weekNumber,totalPoints,stats.mins_played,stats.goals,stats.goal_assist,stats.offtarget_att_assist,stats.pen_area_entries,stats.penalty_won,stats.penalty_save,stats.saves,...,stats.won_contest,stats.ball_recovery,stats.poss_lost_all,stats.penalty_conceded,stats.marca_points,id,name,position,team_id,team_shortName
0,1,6,"[90, 2]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[4, 0]","[11, -1]","[0, 0]","[-1, 1]",53,Jan Oblak,Portero,2,ATM
1,2,3,"[90, 2]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[3, 1]",...,"[0, 0]","[9, 1]","[5, 0]","[0, 0]","[-1, 1]",53,Jan Oblak,Portero,2,ATM
2,3,7,"[90, 2]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[2, 1]",...,"[0, 0]","[9, 1]","[16, -2]","[0, 0]","[-1, 1]",53,Jan Oblak,Portero,2,ATM
3,4,5,"[85, 2]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[4, 2]",...,"[0, 0]","[7, 1]","[16, -2]","[0, 0]","[-1, 2]",53,Jan Oblak,Portero,2,ATM
4,6,1,"[90, 2]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[1, 0]",...,"[0, 0]","[3, 0]","[5, 0]","[0, 0]","[-1, 1]",53,Jan Oblak,Portero,2,ATM


### Create column for each value in stats array (actual, points)

In [182]:
# create new column names based on the original
stats_new_info = []
for col in df_players_stats.columns:
    
    if "stats." not in col:
        continue
    
    stats_new_info.append({
        "current_name": col,
        "new_name_actual": col.replace("stats.", "") + "_actual",
        "new_name_points": col.replace("stats.", "") + "_pts",
    })

# add information to new columns and drop original
for sni in stats_new_info:
    curr_col = sni['current_name']
    new_col1 = sni['new_name_actual']
    new_col2 = sni['new_name_points']

    df_players_stats[[new_col1, new_col2]] = pd.DataFrame(df_players_stats[curr_col].to_list())
    df_players_stats = df_players_stats.drop(columns=curr_col)
    
df_players_stats.head()


,weekNumber,totalPoints,id,name,position,team_id,team_shortName,mins_played_actual,mins_played_pts,goals_actual,...,won_contest_actual,won_contest_pts,ball_recovery_actual,ball_recovery_pts,poss_lost_all_actual,poss_lost_all_pts,penalty_conceded_actual,penalty_conceded_pts,marca_points_actual,marca_points_pts
0,1,6,53,Jan Oblak,Portero,2,ATM,90,2,0,...,0,0,4,0,11,-1,0,0,-1,1
1,2,3,53,Jan Oblak,Portero,2,ATM,90,2,0,...,0,0,9,1,5,0,0,0,-1,1
2,3,7,53,Jan Oblak,Portero,2,ATM,90,2,0,...,0,0,9,1,16,-2,0,0,-1,1
3,4,5,53,Jan Oblak,Portero,2,ATM,85,2,0,...,0,0,7,1,16,-2,0,0,-1,2
4,6,1,53,Jan Oblak,Portero,2,ATM,90,2,0,...,0,0,3,0,5,0,0,0,-1,1


### Average Points per Player

In [183]:
# sort columns
df_players_stats = df_players_stats.sort_values(['id', 'weekNumber'])
df_players_stats.head()

,weekNumber,totalPoints,id,name,position,team_id,team_shortName,mins_played_actual,mins_played_pts,goals_actual,...,won_contest_actual,won_contest_pts,ball_recovery_actual,ball_recovery_pts,poss_lost_all_actual,poss_lost_all_pts,penalty_conceded_actual,penalty_conceded_pts,marca_points_actual,marca_points_pts
5292,1,1,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3,1,0,...,0,0,0,0,0,0,0,0,-1,0
5293,2,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,11,1,0,...,0,0,3,0,0,0,0,0,-1,1
5294,3,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,0,0,0,...,0,0,0,0,0,0,0,0,-1,0
5295,5,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,0,0,0,...,0,0,0,0,0,0,0,0,-1,0
5296,6,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,10,1,0,...,0,0,1,0,0,0,0,0,-1,1


In [184]:
# columns we need from previous week
points_cols = [col for col in df_players_stats.columns if '_pts' in col]
actual_cols = [col for col in df_players_stats.columns if 'actual' in col]

# avg points colums names
# lm = "last match"
avg_points_cols = ['_'.join(['cumavg', col]) for col in points_cols]

df_players_stats['cum_totalPoints'] = df_players_stats.groupby('id')['totalPoints'].apply(lambda x: x.shift(1).cumsum())

# add cumulative average points
df_players_stats['cumavg_totalPoints'] = df_players_stats.groupby('id')['totalPoints'].apply(lambda x: x.shift(1).expanding().mean())

# dropping points of current week
df_players_stats = df_players_stats.drop(columns=(points_cols + actual_cols))



/var/folders/1v/gqg3smmx3d99djll0gyhm06w0000gn/T/ipykernel_12494/2011383607.py:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_players_stats['cum_totalPoints'] = df_players_stats.groupby('id')['totalPoints'].apply(lambda x: x.shift(1).cumsum())
/var/folders/1v/gqg3smmx3d99djll0gyhm06w0000gn/T/ipykernel_12494/2011383607.py:12: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the futu

In [187]:
df_players_stats['cum_totalPoints'] = df_players_stats['cum_totalPoints'].fillna(0)
df_players_stats['cumavg_totalPoints'] = df_players_stats['cumavg_totalPoints'].fillna(0)

df_players_stats.head()

,weekNumber,totalPoints,id,name,position,team_id,team_shortName,cum_totalPoints,cumavg_totalPoints
5292,1,1,1000,Edgar Paul Akouokou,Centrocampista,5,BET,0.0,0.00
5293,2,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,1.0,1.00
5294,3,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.50
5295,5,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.00
5296,6,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,0.75


In [188]:
df_players_stats[['id', 'cumavg_totalPoints']].head()

,id,cumavg_totalPoints
5292,1000,0.00
5293,1000,1.00
5294,1000,1.50
5295,1000,1.00
5296,1000,0.75


## Week Info

In [189]:
weeks_stats_path = './data_json/weeks_stats.json'

### Get data from local JSON

In [190]:
# with open(weeks_stats_path) as f:
#     weeks_stats = json.load(f)

### Get data from API

In [191]:
# get games stats per week
print("Getting week stas...")

weeks = list(range(1, 39))
weeks_stats = []
for w in weeks:
    response = requests.get(f'{HTTPS_DOMAIN_V1}{ROUTE_WEEK(w)}')
    weeks_stats.append(response.json())

    print(w, '- Retrieved week stats for week:', w)
    
# create file 
with open(weeks_stats_path, 'w') as convert_file:
     convert_file.write(json.dumps(weeks_stats))

### Create data frame

In [192]:
# create a flattened list of the week stats
flat_weeks_stats = [
    {'weekNumber': i+1, **item}
    for i, sublist in enumerate(weeks_stats)
        for item in sublist
]


In [193]:
# convert json to data frame
df_weeks_stats = pd.json_normalize(flat_weeks_stats)

df_weeks_stats = df_weeks_stats[['weekNumber',
                                 'localScore', 
                                 'local.id', 
                                 'local.shortName', 
                                 'visitorScore', 
                                 'visitor.id', 
                                 'visitor.shortName']]

In [194]:
df_weeks_stats.head()

,weekNumber,localScore,local.id,local.shortName,visitorScore,visitor.id,visitor.shortName
0,1,3.0,5,BET,0.0,7,ELC
1,1,0.0,9,GET,3.0,2,ATM
2,1,0.0,3,ATH,0.0,33,MLL
3,1,1.0,1,ALM,2.0,15,RMA
4,1,1.0,18,VAL,0.0,28,GIR


## Merge Week Info with Player Stats to add Match Features

In [195]:
# convert to string
df_weeks_stats['local.id'] = df_weeks_stats['local.id'].astype(str)
df_weeks_stats['visitor.id'] = df_weeks_stats['visitor.id'].astype(str)

### Adding next match features

In [196]:
# merge based on local team id
df = df_weeks_stats.merge(
        df_players_stats, 
        how='right',
        right_on=['team_id', 'weekNumber'],
        left_on=['local.id', 'weekNumber'])

# merge based on visitor team id
df = df_weeks_stats.merge(
        df,
        how='right',
        right_on=['team_id', 'weekNumber'],
        left_on=['visitor.id', 'weekNumber'],
        suffixes=('_v', '_l')) # visitor, local

df.head()

,weekNumber,localScore_v,local.id_v,local.shortName_v,visitorScore_v,visitor.id_v,visitor.shortName_v,localScore_l,local.id_l,local.shortName_l,...,visitor.id_l,visitor.shortName_l,totalPoints,id,name,position,team_id,team_shortName,cum_totalPoints,cumavg_totalPoints
0,1,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5,BET,...,7,ELC,1,1000,Edgar Paul Akouokou,Centrocampista,5,BET,0.0,0.00
1,2,1.0,33,MLL,2.0,5,BET,NaN,NaN,NaN,...,NaN,NaN,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,1.0,1.00
2,3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,BET,...,13,OSA,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.50
3,5,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,BET,...,20,VIL,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.00
4,6,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5,BET,...,28,GIR,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,0.75


In [197]:
# adding next match features

# feature describing if played as local
df['curr_match_as_local'] = ~df['localScore_l'].isna()

# feature describing last match opponent
df['curr_match_opponent_id'] = np.where(
    df['curr_match_as_local'], # if
    df['visitor.id_l'], # then
    df['local.id_v'])   # else

df.head()

,weekNumber,localScore_v,local.id_v,local.shortName_v,visitorScore_v,visitor.id_v,visitor.shortName_v,localScore_l,local.id_l,local.shortName_l,...,totalPoints,id,name,position,team_id,team_shortName,cum_totalPoints,cumavg_totalPoints,curr_match_as_local,curr_match_opponent_id
0,1,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5,BET,...,1,1000,Edgar Paul Akouokou,Centrocampista,5,BET,0.0,0.00,True,7
1,2,1.0,33,MLL,2.0,5,BET,NaN,NaN,NaN,...,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,1.0,1.00,False,33
2,3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,BET,...,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.50,True,13
3,5,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5,BET,...,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.00,True,20
4,6,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5,BET,...,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,0.75,True,28


In [198]:
# define condition of redundant columns
reduntant_cols_cond = lambda c: c.startswith("local") or c.startswith("visitor") or c.startswith("lastWeekNumber")

# select redundant columns
redundant_cols = [col for col in df.columns if reduntant_cols_cond(col)]

# drop redundant columns
df = df.drop(columns=redundant_cols)

df.head()

,weekNumber,totalPoints,id,name,position,team_id,team_shortName,cum_totalPoints,cumavg_totalPoints,curr_match_as_local,curr_match_opponent_id
0,1,1,1000,Edgar Paul Akouokou,Centrocampista,5,BET,0.0,0.00,True,7
1,2,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,1.0,1.00,False,33
2,3,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.50,True,13
3,5,0,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,1.00,True,20
4,6,2,1000,Edgar Paul Akouokou,Centrocampista,5,BET,3.0,0.75,True,28


### Adding Last Match Features

In [199]:
# # add last week number to join
# df['lastWeekNumber'] = df.groupby(['id'])['weekNumber'].shift(1)

# df.head()

In [200]:
# # rename column to avoid conflict
# df = df.rename(columns={'weekNumber': 'week'})

# # merge based on local team id
# df = df_weeks_stats.merge(
#         df, 
#         how='right',
#         right_on=['team_shortName', 'lastWeekNumber'],
#         left_on=['local.shortName', 'weekNumber'])

# # merge based on visitor team id
# df = df_weeks_stats.merge(
#         df.drop(columns='weekNumber'),
#         how='right',
#         right_on=['team_shortName', 'lastWeekNumber'],
#         left_on=['visitor.shortName', 'weekNumber'],
#         suffixes=('_v', '_l')) # visitor, local

# # drop week number as it is irrelevant
# df = df.drop(columns='weekNumber')

In [201]:
# # adding last match features

# # feature describing if played as local
# df['last_match_as_local'] = ~df['localScore_l'].isna()

# # feature describing last match opponent
# df['last_match_opponent_id'] = np.where(df['last_match_as_local'], df['visitor.id_l'], df['local.id_v'])

# # feature describing last match goals
# df['last_match_goals'] = np.where(df['last_match_as_local'], df['localScore_l'], df['visitorScore_v'])

# # feature describing last match oponent goals
# df['last_match_oponent_goals'] = np.where(df['last_match_as_local'], df['visitorScore_l'], df['localScore_v'])

# # feature describing if game status
# match_result_conditions = [
#     df['last_match_goals'].isna(),
#     df['last_match_goals'] > df['last_match_oponent_goals'],
#     df['last_match_goals'] < df['last_match_oponent_goals'],
#     df['last_match_goals'] == df['last_match_oponent_goals']
# ]
# match_result_values = [np.nan, 'win', 'loose', 'draw']
# df['last_match_status'] = np.select(match_result_conditions, match_result_values)

# df.head(10)

In [202]:
# # define condition of columns we want to drop
# reduntant_cols_cond = (lambda c: c.startswith("local") or 
#                                  c.startswith("visitor") or 
#                                  c.startswith("lastWeekNumber"))

# # select columns we want to drop
# redundant_cols = [col for col in df.columns if reduntant_cols_cond(col)]

# # drop columns
# df = df.drop(columns=redundant_cols)

# df.head()

In [203]:
# df.shape

In [204]:
# drop the 

In [205]:
df.to_csv('./data_csv/players_full_data_v2.csv', index=False)

## Historical Market Values

### Get data from JSON

In [13]:
# historical_market_values_path_json = './data_json/historical_market_values.json'
# historical_market_values_path_csv = './data_csv/historical_market_values.csv'

In [4]:
# with open(historical_market_values_path_json) as f:
#     historical_market_values = json.load(f)

### Get data from API

In [ ]:
# get historical market value per player
print('Getting players historical values')

# get hmv for each player
historical_market_values = []
for i, p_id in enumerate(players_ids):
    response = requests.get(f'{HTTPS_DOMAIN_V3}{ROUTE_MARKET_VALUE(p_id)}')
    player_hmv = response.json()
    player_hmv = [dict(p, **{'id': p_id}) for p in player_hmv]

    historical_market_values += player_hmv

    print(i+1, '- Retrieved historical market value of id:', p_id)

# create file
with open(historical_market_values_path, 'w') as convert_file:
     convert_file.write(json.dumps(historical_market_values))

len(historical_market_values)

In [12]:
# df_market_values = pd.json_normalize(historical_market_values)

In [6]:
# df_market_values.shape

(114013, 5)

In [7]:
# df_market_values.head()

,lfpId,marketValue,date,bids,id
0,2000053,19000000,2022-07-14T00:00:00+02:00,0,53
1,2000053,19024343,2022-07-15T00:00:00+02:00,0,53
2,2000053,19211522,2022-07-16T00:00:00+02:00,0,53
3,2000053,19633688,2022-07-17T00:00:00+02:00,0,53
4,2000053,19868912,2022-07-18T00:00:00+02:00,0,53


In [172]:
# df_market_values.to_csv(historical_market_values_path_csv, index=False)